# Section 2: Logistic Regression

In this hands-on exercise, we will build an algorithm to identify handwritten digits. Before we design a complex Neural-Networks, we will start developing a simple algorithm. 


### Handwritten digits
You will be working with a dataset that contains handwritten digits.

Load the training Data, in this exercise we will use only small amount of the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
%matplotlib inline

(Xorig, Y),(_,_) = mnist.load_data()
n=2000 # total amount of data to use
Xorig=Xorig[:n]/255; Y=Y[:n]
X = Xorig.reshape(Xorig.shape[0],28*28)
print('Xorig.shape = ',Xorig.shape)
print('X.shape = ',X.shape)
print('Y.shape = ',Y.shape)

The file contains 60000 examples of handwritten digits, we will use only small fraction of it - 2k images. This is a subset of the MNIST handwritten digit dataset (http://yann.lecun.com/exdb/mnist/ ). Each example is $28\times 28$ grayscale image of the digit. The 28 by 28 grid is "unrolled" into a 784-dimensional vector. Each example becomes a single row. This gives 2000 by 784 input matrix $X$.

Let's visualize first $n$ digits, this is defined in function `displayData(data,n)`:

In [ ]:
def getDigit(inputX,i):
    xi = inputX[i]
    return xi

def displayData(X,Y,n,showtrue=True):
    m=len(X)
    inds=np.random.choice(m, n*n)
    outX=np.zeros((n*28,n*28))
    outY=np.zeros((n,n))
    for i in range(n**2):
        row_idx = i//n # equivalent to np.floor(i/n)
        col_inx = i - n*row_idx
        getDigit(X,[inds[i]]).shape
        outX[28*(row_idx):28*(row_idx+1),28*(col_inx):28*(col_inx+1)] = getDigit(X,[inds[i]])
        outY[row_idx,col_inx]=Y[inds[i]]
    plt.imshow(outX)   
    if showtrue:
        print('The labels are:')
        print(' '+np.array2string(outY.astype(int), precision=2, separator=' ')[1:-1])

Run next cell to visualize the data

In [ ]:
n = 10  # to display nxn matrix of n*n random digits from the input data
displayData(Xorig,Y,n)

Our goal will be the identification of the handwritten digits!

Before developing the algorithm, let's take a look on the average value of pixels

Lets draw a scatter plot, for two types of digits $1$ and $8$:

In [ ]:
#filter first n digits with Y=1 and 8
n=10
X1=Xorig[np.where(Y==1)][:n*n]
X8=Xorig[np.where(Y==8)][:n*n]
Y1=Y[np.where(Y==1)][:n*n]
Y8=Y[np.where(Y==8)][:n*n]
plt.figure(figsize=(15,7))
plt.subplot(221)
displayData(X1,Y1,n,False)
plt.subplot(222)
displayData(X8,Y8,n,False)
plt.subplot(223)
plt.hist(np.sum(X1,axis=(1,2))/784,100,(0,0.25),label='mean = %2.2f'%np.mean(np.sum(X1,axis=(1,2))/784))
plt.subplot(223).legend(); plt.xlabel('pixel average');
plt.subplot(224)
plt.hist(np.sum(X8,axis=(1,2))/784,100,(0,0.25),label='mean = %2.2f'%np.mean(np.sum(X8,axis=(1,2))/784))
plt.subplot(224).legend();  plt.xlabel('pixel average');
plt.show()


Can you find a function that can distinguish between two digits?

Up to now, we considered a simple problem of fitting a linear function $y = ax + b$

Here we are dealing a clasification problem - result is **Yes/No** (where $y = 1$ is positive class, and $y = 0$ is negative class).

The output label, then will have next form $y =  \left\{ \begin{eqnarray} 1 & \text{ for } x=N \\ 0 & \text{ for } x\neq N \\ \end{eqnarray} \right. $

The output function $h_\theta(x)$ should be in the region between $0$ and $1$.

### Using "Logistic function" as  an activation function

We need to find a function, such that the output will be in the range between 0 and 1.

A good candidate is "logistic function" or "sigmoid function" (https://en.wikipedia.org/wiki/Sigmoid_function). 

<br>
<center><font size="5">  $g(z) = \frac{1}{1 + e^{-z}}$ </font> </center> 

Let's define and visualize the function.

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
x = np.linspace(-10, 10, num=100)
y = sigmoid(x)
plt.plot(x,y,'r-')
plt.ylabel('logistic function: g(z)',fontsize=20); plt.xlabel('z',fontsize=20)
plt.show()

### Logistic cost function

As we did in section 1, let's define a cost function in the following way:

<center><font size="5">  
$ J(\theta)  =  Cost(h_\theta(x),y) = \left\{ \begin{eqnarray} -\log(h_\theta(x)) & \text{ if } y = 1 \\ -\log(1 - h_\theta(x)) & \text{ if } y = 0 \\ \end{eqnarray} \right. $
</font> </center> 

Let's draw it:

In [ ]:
plt.plot(y,-np.log(y),'r-',y,-np.log(1-y),'b-')
plt.legend(['-log(h)','-log(1-h)']); plt.xlabel(r'$h_\theta (\theta\cdot x)$')
plt.ylabel(r'J($\theta$)'); plt.show()

To start the learning (gradient descent) process to minimize the cost function, we should compute derivatives of the cost with respect to $\theta$. Fortunately, with the current definition of the cost function, the gradient has a simple form of:

<center><font size="5">  
$ \frac{dJ}{d\theta_j} = \frac{dJ(\theta | x,y)}{d\theta_j} = \left(h_\theta(x) - y\right)x_j $
</font> </center> 

Lets define both of them in `CostFunction()`:

In [ ]:
def CostFunction(theta, X, Y):
    """
    Implementation of the cost function for logistic regresion
    
    Arguments:
    theta -- input vector of weights of shape (nf+1, 1)
    intputX -- input data matrix of shape (m,n)
    Y -- input labels of the data of shape (m,1) that contains {0,1}
    
    Returns:
    J -- cost function for given input data and given weights
    grad - vector of the gradients of the cost function with respect to each weight (shape - nf+1,1)
    """    
    invm = 1 / Y.shape[0]
    Xtmp = np.insert(X,0,1,axis=1)
    Xt = np.dot(Xtmp,theta)
    ht = sigmoid(Xt)
    J = -invm*(np.dot(Y.T,np.log(ht)) + np.dot(1-Y.T,np.log(1-ht)))
    #grad = np.sum((ht - Y)*Xtmp*invm,axis=0,keepdims=True)
    grad = invm*np.dot((ht - Y).T,Xtmp).T

    return J, grad

Now let's implement the gradient descent algorithm, by computing gradients for several iterations:

In [ ]:
def TrainModel(initial_weights, X, Y, epoch = 10, learning_rate=0.001, vebrose=True):
    
    theta = initial_weights
    Jvec=np.zeros((epoch,1))
    for i in range(epoch):
        
        J, grad = CostFunction(theta,X,Y)
        Jvec[i]=J
        
        theta = theta - learning_rate*grad
        if vebrose:
            print('iteration n = ',i,'Cost funcion = ',J)
        
    return theta,Jvec

Before continuing, let's validate that the implimination is exact. For that propuse, we can compute gradient numerically for a given input vector $\vec{\theta}$ using the formula

$$ \frac{dJ}{d\theta_j} = \frac{J(\theta + \varepsilon_j) - J(\theta - \varepsilon_j)}{2|\varepsilon|} $$ 

where $\vec{\varepsilon_j} = (0,0,...,\varepsilon,0,...,0)$, with non zero value on entry $j^\text{th}$.


In [ ]:
def initialize_theta(m,n):
    return np.random.rand(n+1,1)/float(m)
    

def CheckGradient(X,Y):
    m, nf = X.shape
    epsilon=1e-5
    theta=initialize_theta(m,nf)
    grad_diff=np.zeros((nf+1,1))
    th,grad = CostFunction(theta,X,Y)
    for i in range(nf+1):
        theta_plus=np.copy(theta); theta_minus=np.copy(theta);
        theta_plus[i] = theta_plus[i] + epsilon
        theta_minus[i] = theta_minus[i] - epsilon
        Jplus,grp = CostFunction(theta_plus,X,Y)
        Jminus,grm = CostFunction(theta_minus,X,Y)
        dJ = 0.5*(Jplus - Jminus)/epsilon
        grad_diff[i]=dJ
        if i<50 and grad_diff[i]:
            print('check gradient',i,': numerical = ',grad_diff[i],', computed = ',grad[i],' diff/eps=',(grad_diff[i]-grad[i])/grad_diff[i])
    #grad_diff = grad_diff - grad
    #print(grad)
    #print(grad_diff)
        

Let's check if the gradient descent is implemented properly:

In [ ]:
y4 = np.zeros((len(Y),1));
y4[np.where(Y==4)]=1
CheckGradient(X,y4)

Once we defined a model, let's train the algorithm to identify a specific digit. We will change the representation of the labeled data $Y$ to be $\{0,1\}$ if it equals the correct number. Let's compute for $y=4$:

In [ ]:
#initialize weights
theta = initialize_theta(X.shape[0],X.shape[1])

#Train model to get the trained weights for different learning rate:
print('Traning for learning rate = 1')
[theta4lr1,Jlr1] = TrainModel(theta,X,y4,1000,1,False)
print('Traning for learning rate = 0.1')
[theta4lr2,Jlr2] = TrainModel(theta,X,y4,1000,0.1,False)
print('Traning for learning rate = 0.01')
[theta4lr3,Jlr3] = TrainModel(theta,X,y4,1000,0.01,False)
print('Traning for learning rate = 0.001')
[theta4lr4,Jlr4] = TrainModel(theta,X,y4,1000,0.001,False)

Let's test the algorithm convergence for different training rates. 

In [ ]:
iteration=range(0,1000)
plt.plot(iteration,Jlr1[iteration],'b',iteration,Jlr2[iteration],'g',iteration,Jlr3[iteration],'r',iteration,Jlr4[iteration],'c')
plt.legend([r'$\alpha$ = 1.000',r'$\alpha$ = 0.100',r'$\alpha$ = 0.010',r'$\alpha$ = 0.001'])
plt.show()


The result is a trained vector $\theta$, such that $h_{\theta}(x)$ is maximized when $y = 4$

Let's choose threshold value, first define Activation function, and prediction:

- Activation function: $a(\theta,X)=\sigma\left(X\cdot\theta\right)\in(0,1)$
- prediction function: $ p(x_i) = \left\{ \begin{eqnarray} 1 & \text{ if } a(\theta,x_i) > \text{threshold} \\ 0 & \text{ otherwise } \\ \end{eqnarray} \right. $
- accuracy function: number of correctly predicted entries / total number of different entries (in %)


In [ ]:
def getActivation(theta,intputX):
    m = intputX.shape[0]
    Xtmp = np.insert(intputX,0,1,axis=1)
    Xt = np.dot(Xtmp,theta)
    return sigmoid(Xt)

def predict(A,th):
    m=A.shape[0]
    y = np.zeros((m,1))
    y[A > th] = 1
    return y

def accuracy(yhat,y):
    m=y.shape[0]
    return np.sum(yhat==y)/m*100.

Calculate accuracy of the previously trained model (recall we obtain variable `theta4`), let's define a threshold of 0.5, and calculate how well we predict  digits

In [ ]:
A4 = getActivation(theta4lr1,X)
yhat = predict(y4,0.5)
print('accuracy = ',accuracy(yhat,y4))

Let's find a threshold, that maximizes accuracy for the given example:

In [ ]:
threshold_best=0
acc_best=0
threshold=np.arange(0.2, 1.0, 0.01)
acc=np.zeros(threshold.shape)
for i in range(threshold.shape[0]):
    yhat = predict(A4,threshold[i])
    acc[i] = accuracy(yhat,y4)
    if acc_best<acc[i]: acc_best=acc[i]; threshold_best=threshold[i]
fig, ax = plt.subplots()
ax.plot(threshold, acc)

ax.set(xlabel='threshold (s)', ylabel='accuracy (%)',
       title='Optimization of the threshold')
ax.grid()
print('best threshold is %2.2f that gives accuracy of %2.2f' % (threshold_best,acc_best)+'%')

Rembmer, we have 10 digits, while we only trained the model for $y=4$. Let's repeat the exersize for all digits, and check if it improves the estimation:

In [ ]:
def runModel(X,Y,i):
    m, nf = X.shape
    theta = initialize_theta(m,nf)
    yi=np.zeros((m,1)); yi[Y==i]=1
    theta, _ = TrainModel(theta,X,yi,100,1,False)
    A = getActivation(theta,X)
    Yhat = predict(A,0.5)
    accuracy = np.sum(Yhat==yi) / m * 100.0 # in percent
    indx = np.where(Yhat!=yi)
    return theta, Yhat, yi, A, accuracy, indx


In [ ]:
nlabels=10
yN=[];Theta=[]; accuracy=[]; indx=[]; Yhat = []; Ai=[]
for i in range(nlabels):
    theta, Yh, yi, a, acc, ind = runModel(X,Y,i);
    yN.append(yi)
    Theta.append(theta)
    accuracy.append(acc)
    Ai.append(a)
    indx.append(ind[0])
    Yhat.append(Yh)
    print('done for',i,' accuracy = ',acc)
    


We see high accuracy for training separately each one of the digits. In the combined method, we can prob, which of the digits has the highest probability (highest activation, softmax activation).

Let loon on a specific example where the algorithm failed to identify correctly digit 4:
the indeces of incorrectly identified digits are stored in list `indx`:

In [ ]:
wrongindex = np.random.choice(indx[4],1)[0]
print('The correct gidit is '+str(Y[wrongindex]))
for i in range(10):
    print('for digit = '+str(i)+' the value, the predicted activation value is = ',Ai[i][wrongindex])

plt.imshow(getDigit(Xorig,wrongindex))
plt.show()

## Neural networks

In this section, we will implement a basic neural network to identify digits. Although logistic regression does a pretty good job, we will see in the further example, that for more sophisticated image classification, logistic regression will be not that precise.

Until now we define the task as follows:
- for data $X$ and labels $Y$, we defined a function $F(X,\theta)$, where $\theta$ are free parameters of the problem
- To find the optimal function, we defined a cost function $J(\theta)$ that holds $\forall x$ $F(x,\theta)\approx Y$
- we used gradient descent to minimize the function with respect to free parameters $\theta$
- This process is called "learning process of the ML algorithm"

Today, there are existing packages, that do this job efficiently and we use the one for this propose.

### Keras

Programing with keras is very easy. First we need to load relevant libraries


In [ ]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras import losses
from keras import optimizers

#visualization of the keras model
from IPython.display import Image
from keras.utils import plot_model

### 2 - Building a model

Keras is very good for rapid prototyping. In just a short time you will be able to build a model that achieves outstanding results.

Here is an example of a model in Keras:

```python
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Make activation function for the input layer
    X = Dense(1, activation='sigmoid', name='fc')(X_input)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='MyFirstModel')
    
    return model
```

In [ ]:
def MyFirstModel(input_shape):
    """
    Implementation of the MyFirstModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape, name='Image')
    
    # DEnse X creates a FULLYCONNECTED layer
    X = Dense(1, activation='sigmoid', name='OutputLayer')(X_input)   
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='MyFirstModel')    
    
    ### END CODE HERE ###
    
    return model

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).

1. Creating a model:

In [ ]:
MyFirstModel = MyFirstModel((X.shape[1:]))

2. Compile the model to configure the learning process. Choose the 3 arguments of `compile()` wisely. Here we will use "ADAM" (adaptive moment) optimizer, which is a more advanced version of gradient descent

In [ ]:
adam=optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
MyFirstModel.compile(optimizer = adam, loss='binary_crossentropy', metrics = ['accuracy'])

3. Train the model. Choose the number of epochs and the batch size. Train the example only for $y=4$

For training use only half of the data ($n=1500$)

In [ ]:
miniX=X[:1500]; miniY=Y[:1500]
y4=np.zeros((miniX.shape[0],1)); y4[miniY==4]=1
history = MyFirstModel.fit(x = miniX,y = y4, epochs = 100, batch_size = 128, verbose=0)

Note that if you run `fit()` again, the `model` will continue to train with the parameters it has already learnt instead of reinitializing them.

4. Test/evaluate the model.
The testing will be done on a sample that the model didn't saw:

In [ ]:
#select new dataset:
miniX_test=X[1500:]; miniY_test=Y[1500:]
#develop the model for Y=4, therefore change labels to 1 or 0 if Y=4 of else respectively 
y4_test = np.zeros((500,1));
y4_test[np.where(miniY_test==4)]=1

In [ ]:
preds_train = MyFirstModel.evaluate(x = miniX, y = y4, batch_size=miniX.shape[1])
preds_test = MyFirstModel.evaluate(x = miniX_test, y = y4_test, batch_size=miniX_test.shape[1])

In [ ]:
print("Loss(train) = " + str(preds_train[0])+", Loss(test) = " + str(preds_test[0]))
print("Train Accuracy = " + str(preds_train[1])+" Test Accuracy = " + str(preds_test[1]))

### Other useful functions in Keras

Two other basic features of Keras that you'll find useful are:
- `model.summary()`: prints the details of your layers in a table with the sizes of its inputs/outputs
- `plot_model()`: plots your graph in a nice layout. You can even save it as ".png" using SVG() if you'd like to share it on social media ;). It is saved in "File" then "Open..." in the upper bar of the notebook.

Run the following code.

In [ ]:
MyFirstModel.summary()

In [ ]:
plot_model(MyFirstModel, to_file='MyFirstModel.png', show_shapes=True, show_layer_names=False)
Image("MyFirstModel.png")